# DC Voltage Source Setup

## Introduction
In this tutorial we will show how to create and use the spinQICK voltage source object. SpinQICK tightly integrates the DC source within experiments and saves the dc voltage state whenever it saves data, so setting up a compatible instrument driver is necessary for correct operation.  

In [ ]:
from spinqick.helper_functions import hardware_manager

## Necessary DAC and trigger settings

The following variables need to be set using the `DacSettings` located in `settings.py`:

    t_min_slow_dac: float = 3.0
    trig_length: float = 0.2
    trig_pin: int = 0

These pertain to the trigger signal which we send from the RFSoC to the DAC.  The `trig_pin` is a digital output pin on the RFSoC which you must connect to your DC source. The pins are listed in the bottom of your QickConfig printout under "6 digital output pins:", and the physical location of these pins depends on if you are using the RF board/QICKBox firmware or not, with the standard QICK firmware having these triggers mapped to the PMOD0 header (marked 22 [here](https://docs.amd.com/r/en-US/ug1390-zcu216-eval-bd/Board-Component-Location?section=kyi1567617969281__fig_xmm_ttm_1jb)).  If you are not using the QICKBox and RF board firmware, we recommend using a [level shifter](https://digilent.com/shop/pmod-lvlshft-logic-level-shifter/?srsltid=AfmBOoqZodUKJkK6xvxAk7vgOS6NISjlLeNHoWDSeB-TueM1wp54cUVR) to buffer this trigger output.

## Setting up a DC source driver

SpinQICK experiments take an initialized DC source object as their arguement.  The DC source class needs to be formatted in a specific way to work with spinQICK, which is given by the `VoltageSource` Protocol in `spinqick.helper_functions.hardware_manager`. The user driver methods will need to map to those in the `VoltageSource` class to correctly function. Below is an example of a dummy voltage source driver class which you could populate with calls to your own voltage source:

In [ ]:
class SlowVsource:
    def __init__(self, address: str):
        self.open(address)
        self.voltages = [0.2, 0.5, 0.1, 1, 1.2, 0.4, 0.9, 0.8, 1]

    def open(self, address: str):
        print("now open")

    def close(self):
        print("now closed")

    def get_voltage(self, ch: int) -> float:
        return self.voltages[ch]

    def set_voltage(self, ch: int, volts: float):
        self.voltages[ch] = volts
        print(f"ch {ch} is set to {volts} volts")

    def set_sweep(
        self,
        ch: int,
        start: float,
        stop: float,
        length: float,
        num_steps: int,
    ):
        print(
            f"ch {ch} is set to sweep {num_steps} steps from {start} to {stop} in {length} seconds"
        )

    def trigger(self, ch: int):
        print("trigger set")

    def arm_sweep(self, ch: int):
        return f"sweep armed on ch {ch}"

    def test_func(self):
        print("this is a test")
        pass


# initialize voltage source
vs = SlowVsource("12345")
vs.get_voltage(1)

Now you can use the DCSource class in spinQICK.  This class uses your hardware config, and does all the mapping of gate names to channel numbers, and gate voltages to DAC bias voltages for you.  This is the class that the rest of spinQICK uses to interact with your DC voltage source as well.

In [ ]:
dc_source = hardware_manager.DCSource(vs)
dc_source.program_ramp(vstart=0, vstop=1, tstep=15, nsteps=10, gate="P1")

The dc_source object will print a dictionary of gate name/gate voltage values which is useful for saving and restoring the setup of your device.

In [ ]:
dc_source.all_voltages